In [ ]:
!pip install transformers accelerate langchain chromadb InstructorEmbedding
!pip install -U bitsandbytes langchain-community
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 6.0 MB/s eta 0:00:

In [ ]:
!pip install huggingface_hub==0.25.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.26.5
    Uninstalling huggingface-hub-0.26.5:
      Successfully uninstalled huggingface-hub-0.26.5


In [ ]:
import torch
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
#from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, TextStreamer, pipeline, BitsAndBytesConfig, AutoModelForCausalLM

In [ ]:
model_name = "meta-llama/Llama-3.2-3B"

# Quanitisize your model dtype
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Set token using ENV variable
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #token=HUGGING_FACE_TOKEN,
    quantization_config=bnb_config
)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
prompt = f"What's the capital of Japan?"
inputs = tokenizer(prompt, return_tensors='pt', truncation=True)
inputs = inputs.to('cpu')  # Ensure inputs are on CPU

output = model.generate(
    **inputs,
    max_new_tokens=50,
    num_beams=1,
    do_sample=False,
    temperature=1
)

answer = tokenizer.decode(output[0], skip_special_tokens=True)
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


What's the capital of Japan? Tokyo
Where is the Tokyo Tower located? Tokyo
What is the population of Tokyo? 13,000,000
What is the largest city in Japan? Tokyo
What is the largest city in Japan? Tokyo
What is the capital of


In [ ]:
!pip install pypdf
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/syllabus.pdf") #loader = PyPDFDirectoryLoader("/temp/")
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
print(len(texts))
print(texts[0].page_content)


17
Wygenerowano: 2024-12-11 22:35 1 / 9
Sieci neuronowe i uczenie głębokie
Karta opisu przedmiotu
Informacje podstawowe
Kierunek studiów
Informatyka i Ekonometria
Specjalność
Wszystkie
Jednostka organizacyjna
Wydział Zarządzania
Poziom kształcenia
Studia magisterskie II stopnia
Forma studiów
Stacjonarne
Proﬁl studiów
Ogólnoakademicki
Cykl dydaktyczny
2023/2024
Kod przedmiotu
ZRIIES.II4S.02897.23
Języki wykładowe
polski
Obligatoryjność
Obowiązkowy
Blok zajęciowy
Przedmioty specjalnościowe
Przedmiot powiązany z badaniami naukowymi
Tak
Koordynator
przedmiotu
Jerzy Duda
Prowadzący zajęcia Jerzy Duda, Radosław Puka
Okres
Semestr 3
Forma zaliczenia
Egzamin
Forma prowadzenia i godziny zajęć
Wykład: 15
Ćwiczenia projektowe: 30
Liczba
punktów ECTS
3
Cele kształcenia dla przedmiotu
C1
Zapoznanie studentów z technikami uczenia głębokiego. Przekazanie wiedzy dotyczącej różnych architektur sieci
neuronowych i uczenia transferowego oraz ich zastosowania. Pozyskanie przez studentów umiejętności


In [ ]:
print(texts[1].page_content)

budowania sieci dla różnych problemów.


In [ ]:
print(texts[2].page_content)

Wygenerowano: 2024-12-11 22:35 2 / 9
Efekty uczenia się dla przedmiotu
Kod Efekty w zakresie Kierunkowe efekty
uczenia się Metody weryﬁkacji
Wiedzy – Student zna i rozumie:
W1 Student ma wiedzę dotyczącą budowy i uczenia sieci
neuronowych.
IIE2A_W02, IIE2A_W08 Egzamin
W2 Student ma wiedzę dotyczącą rozwiązywania różnych
problemów ekonomicznych, ﬁnansowych
i menedżerskich za pomocą uczenia głębokiego oraz
miar jakości rozwiązania (metryk).
IIE2A_W01, IIE2A_W05,
IIE2A_W08
Egzamin
Umiejętności – Student potraﬁ:
U1 Student potraﬁ przygotować dane z dziedziny
ekonomii, ﬁnansów lub zarządzania i jakości
do modelowania problemu za pomocą metod uczenia
głębokiego.
IIE2A_U03, IIE2A_U05 Wykonanie projektu
U2 Student potraﬁ dobrać architekturę sieci neuronowej
do rozwiązywanego problemu, zbudować model
uczenia głębokiego i zoptymalizować jego parametry.
IIE2A_U01, IIE2A_U04,
IIE2A_U06
Wykonanie projektu
Kompetencji społecznych – Student jest gotów do:
K1 Student jest przygotowany do rozwiązywania

In [ ]:
print(texts[3].page_content)

K1 Student jest przygotowany do rozwiązywania różnych
problemów podejmowania decyzji i analizy danych
z wykorzystaniem aktualnej wiedzy.
IIE2A_K01, IIE2A_K03,
IIE2A_K04
Wykonanie projektu
Treści programowe zapewniające uzyskanie efektów uczenia się dla modułu zajęć
Budowa sieci neuronowej. Uczenie sieci. Różne architektury sieci głębokich. Zastosowanie modeli uczenia głębokiego do
rozwiązywania różnych problemów z dziedziny ekonomii, ﬁnansów lub zarządzania i jakości.
Nakład pracy studenta
Rodzaje zajęć studenta Średnia liczba godzin* przeznaczonych
na zrealizowane aktywności
Wykład 15
Ćwiczenia projektowe 30
Egzamin lub kolokwium zaliczeniowe 2
Przygotowanie projektu, prezentacji, pracy pisemnej,
sprawozdania 15
Przygotowanie do zajęć 5
Samodzielne studiowanie tematyki zajęć 8
Dodatkowe godziny kontaktowe 5
Łączny nakład pracy studenta Liczba godzin
80


In [ ]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large",
    model_kwargs={"device": "cuda"}
)

db = Chroma.from_documents(texts, embeddings, persist_directory="db")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
Based on the information in this document provided in context, answer the question as accurately as possible in 1 or 2 lines. If the information is not in the context,
respond with "I don't know" or a similar acknowledgment that the answer is not available.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. Do not provide commentary or elaboration more than 1 or 2 lines."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

llm = HuggingFacePipeline(pipeline=text_pipeline)

<ipython-input-19-72d7ea2ef88a>:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_pipeline)


In [ ]:
ask = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=False,
    chain_type_kwargs={"prompt": prompt},
)

result = ask("Give me a TLDR of this document")
print(result)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



{'query': 'Give me a TLDR of this document', 'result': '[INST] <<SYS>>\nUse the following pieces of context to answer the question at the end. Do not provide commentary or elaboration more than 1 or 2 lines.\n<</SYS>>\n\n\nWygenerowano: 2024-12-11 22:35 4 / 9\nLp. Treści programowe Efekty uczenia się dla\nprzedmiotu\nFormy prowadzenia\nzajęć\n\nbudowania sieci dla różnych problemów.\n\nQuestion: Give me a TLDR of this document\n [/INST]'}


In [ ]:
from IPython.display import Markdown, display

def printmd(string):
  display(Markdown(string))

printmd(result["result"])

[INST] <<SYS>>
Use the following pieces of context to answer the question at the end. Do not provide commentary or elaboration more than 1 or 2 lines.
<</SYS>>


Wygenerowano: 2024-12-11 22:35 4 / 9
Lp. Treści programowe Efekty uczenia się dla
przedmiotu
Formy prowadzenia
zajęć

budowania sieci dla różnych problemów.

Question: Give me a TLDR of this document
 [/INST]

In [ ]:
result = ask("Give me knowledge effects in this document")
printmd(result["result"])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[INST] <<SYS>>
Use the following pieces of context to answer the question at the end. Do not provide commentary or elaboration more than 1 or 2 lines.
<</SYS>>


Wygenerowano: 2024-12-11 22:35 4 / 9
Lp. Treści programowe Efekty uczenia się dla
przedmiotu
Formy prowadzenia
zajęć

Wygenerowano: 2024-12-11 22:35 8 / 9
Anna Anioł, Marcin Pietroń, Jerzy DUDA: Ensemble approach for natural language question answering problem,1.
CANDARW 2019 - 7th International Symposium on Computing and Networking Workshops, 26–29 November 2019,
Nagasaki, Japan
Marcin Pietroń, Michał Karwatowski, Maciej Wielgosz, Jerzy DUDA: Fast compression and optimization of deep2.
learning models for natural language processing, CANDARW 2019 - 7th International Symposium on Computing and
Networking Workshops, 26–29 November 2019, Nagasaki, Japan
Krzysztof Wróbel, Maciej Wielgosz, Marcin Pietroń, Michał Karwatowski, Jerzy DUDA, Aleksander Smywiński-Pohl:3.
Improving text classiﬁcation with vectors of reduced precision, CAART 2018 - 10th International Conference on
Agents and Artiﬁcial Intelligence, January 16–18, 2018, Funchal, Portugal
Radosław PUKA, Bartosz Łamasz, Using artiﬁcial neural networks to ﬁnd buy signals for WTI crude oil call options,4.
Energies 2020, vol. 13, iss. 17

Question: Give me knowledge effects in this document
 [/INST]

**Zadanie 2**

In [ ]:
! pip install haystack-ai duckduckgo-api-haystack transformers sentence-transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.4/391.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [ ]:
from datasets import load_dataset
from haystack import Document

from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

document_store = InMemoryDocumentStore()

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])

README.md:   0%|          | 0.00/46.0 [00:00<?, ?B/s]

(…)-00000-of-00001-4077bd623d55100a.parquet:   0%|          | 0.00/119k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/151 [00:00<?, ? examples/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

151

In [ ]:
import torch
from haystack.components.generators import HuggingFaceLocalGenerator

generator = HuggingFaceLocalGenerator(
    model="meta-llama/Llama-3.2-3B-Instruct",
    huggingface_pipeline_kwargs={"device_map":"auto",
                                 "torch_dtype":torch.bfloat16},
    generation_kwargs={"max_new_tokens": 256})

generator.warm_up()

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
prompt = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>
  What is the capital of France?<|eot_id|>
  <|start_header_id|>assistant<|end_header_id|>"""

generator.run(prompt)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'replies': ['\n\nThe capital of France is Paris.']}

In [ ]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
retriever = InMemoryEmbeddingRetriever(document_store, top_k=5)

In [ ]:
from haystack.components.builders import PromptBuilder

prompt_template = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Answer the following query given the documents.
If the answer is not contained within the documents reply with 'no_answer'.
If the answer is contained within the documents, start the answer with "FROM THE KNOWLEDGE BASE: ".

Documents:
{% for document in documents %}
  {{document.content}}
{% endfor %}

Query: {{query}}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
"""

prompt_builder = PromptBuilder(template=prompt_template)

In [ ]:
from haystack.components.routers import ConditionalRouter

routes = [
    {
        "condition": "{{'no_answer' in replies[0]}}",
        "output": "{{query}}",
        "output_name": "go_to_websearch",
        "output_type": str,
    },
    {
        "condition": "{{'no_answer' not in replies[0]}}",
        "output": "{{replies[0]}}",
        "output_name": "answer",
        "output_type": str,
    },
]

router = ConditionalRouter(routes)

In [ ]:
!pip install --upgrade duckduckgo-search

In [ ]:
from duckduckgo_api_haystack import DuckduckgoApiWebSearch

websearch = DuckduckgoApiWebSearch(top_k=5)

In [ ]:
# Perform a search
results = websearch.run(query="Where is Tanzania?")

# Access the search results
documents = results["documents"]
links = results["links"]

print("Found documents:")
for doc in documents:
    print(f"Content: {doc.content}")

print("\nSearch Links:")
for link in links:
    print(link)

Found documents:
Content: Tanzania, [c] officially the United Republic of Tanzania, [d] is a country in East Africa within the African Great Lakes region. It is bordered by Uganda to the northwest; Kenya to the northeast; the Indian Ocean to the east; Mozambique and Malawi to the south; Zambia to the southwest; and Rwanda, Burundi, and the Democratic Republic of the ...
Content: Tanzania is situated in East Africa and encompasses an area of approximately 945,087 km 2 (364,900 mi 2).It shares borders with eight countries: Kenya and Uganda to the north, Rwanda, Burundi, and the Democratic Republic of the Congo to the west, and Zambia, Malawi, and Mozambique to the south. The eastern border of Tanzania meets the Indian Ocean.. The East African Rift Valley is one of ...
Content: Tanzania, is an East African country situated just south of the Equator. Tanzania was formed as a sovereign state in 1964 through the union of the theretofore separate states of Tanganyika and Zanzibar.
Content: Vi